# Compare Benchmark Runs
This notebook demonstrates how you can analyze the differences between two benchmark runs of the same benchmark and find the tests that differ the most, which probably means that they require further analysis to figure out why they changed.

Several projects exist in the `examples` folder, but this notebook assumes we are working on the
JVM part of the `kotlin-multiplatform` project. But the same approach can be used for the other projects.

First, you need to run the benchmark twice. This can be done by running these commands from the root of the project:

```shell
> ./gradlew :examples:kotlin-multiplatform:jvmBenchmark
> ./gradlew :examples:kotlin-multiplatform:jvmBenchmark
```

Once it is completed, run this notebook, and it will automatically find the latest result.

In [22]:
%use serialization, dataframe, kandy

In [23]:
import java.nio.file.Files
import java.nio.file.attribute.BasicFileAttributes
import kotlin.io.path.exists
import kotlin.io.path.forEachDirectoryEntry
import kotlin.io.path.isDirectory
import kotlin.io.path.listDirectoryEntries
import kotlin.io.path.readText

// Find latest result file, based on the their timestamp.
val runsDir = notebook.workingDir.resolve("kotlin-multiplatform/build/reports/benchmarks/main")
val outputFiles = runsDir.listDirectoryEntries()
    .filter { it.isDirectory() }
    .sortedByDescending { dir -> Files.readAttributes(dir, BasicFileAttributes::class.java).creationTime() }
    .subList(0, 2)
    .map { it.resolve("jvm.json") }

In [24]:
// Convert to "typed" JSON
val newRun = outputFiles.first().readText().deserializeJson()
val oldRun = outputFiles.last().readText().deserializeJson()

In [25]:
// Convert to DataFrames for easier processing. As there is not "id" keys for the benchmark, we invent one by just
// assigning the test row index as their "primary key". We could attempt to use the benchmark name and param values,
// but that is complicated by how paramers are represented in the JSON file. So, since we assume that the two files
// are equal using row index should be safe.
val oldDf = oldRun.toDataFrame().addId("rowIndex")
val newDf = newRun.toDataFrame().addId("rowIndex")

In [26]:
val combinedData = oldDf.innerJoin(newDf) { rowIndex }
// Un-commont this to see the intermediate dataframe:
// combinedData

In [27]:
import kotlinx.serialization.json.encodeToJsonElement

// Reduce the combined data into the exact format we need
val resultData = combinedData.mapToFrame {
    "name" from { it.benchmark }
    "params" from {
        if (it.params != null) {
            Json.decodeFromJsonElement<Map<String, String?>>(Json.encodeToJsonElement(it.params))
                .entries
                .filter { it.value != null }
                .sortedBy { it.key }
                .map { "${it.key}=${it.value}" }
                .joinToString(",")
        } else {
            ""
        }
    }
    "mode" from { it.mode } // "avgt" or "thrpt"
    "unit" from { it.primaryMetric.scoreUnit }
    "runOld" {
        "score" from { it.primaryMetric.score }
        "range" from { it.primaryMetric.scoreConfidence[0]..it.primaryMetric.scoreConfidence[1] }
    }
    "runNew" {
        "score" from { it.primaryMetric1.score }
        "range" from { it.primaryMetric1.scoreConfidence[0]..it.primaryMetric1.scoreConfidence[1] }
    }
}
// Un-commont this to see the intermediate dataframe:
// resultData

In [28]:
// Flatten the data so it is easier to plot
val mergedData = resultData.unfold { runOld and runNew }.flatten()
// Un-commont this to see the intermediate dataframe:
// mergedData

In [29]:
// Before plotting the data, we calculate the change between the two runs. This is saved
// in "scoreDiff". This is done slightly different depending on the test mode:
//
// - "avgt": For the average time we use "oldScore - newScore", so improvements in the
//   benchmark result in positive numbers.
// - "thrpt": For throughput, we use "newScore - oldScore", so improvements here also
//   result in positive numbers.
//
// We also normalize this value as a percentage change from `scoreOld`. This is saved in
// "scoreDiffPercentage".
val plotData = mergedData
    .add("diffScore") {
        when (mode) {
            "avgt" -> score - score1
            "thrpt" -> score1 - score
            else -> error("Unknown mode: $mode")
        }
    }
    .add("diffScorePercentage") {
        (get("diffScore") as Double) * 100.0 / score
    }
    .add("testLabel") {
        if (params.isNullOrBlank()) {
            name
        } else {
            "$name\n[$params]"
        }
    }
    .add("barColor") {
        val value = get("diffScorePercentage") as Double
        if (value < 0.0) "neg" else "pos"
    }
plotData

name,params,mode,unit,score,range,score1,range1,diffScore,diffScorePercentage,testLabel,barColor
test.InheritedBenchmark.baseBenchmark,,thrpt,ops/s,1053176.103332,1014444.6854321724..1091907.5212313458,999715.573058,974954.2809652109..1024476.8651513313,-53460.530273,-5.076125,test.InheritedBenchmark.baseBenchmark,neg
test.InheritedBenchmark.inheritedBenc...,,thrpt,ops/s,140537144.603648,1.3690787959174788E8..1.4416640961554...,132519854.002753,1.2936737342079675E8..1.3567233458470...,-8017290.600895,-5.704748,test.InheritedBenchmark.inheritedBenc...,neg
test.ParamBenchmark.mathBenchmark,"data=1,text=a ""string"" with quotes,va...",thrpt,ops/ms,208887.080329,207898.89918554123..209875.26147208977,196902.036134,191654.7423267418..202149.32994077116,-11985.044195,-5.737571,test.ParamBenchmark.mathBenchmark [da...,neg
test.ParamBenchmark.mathBenchmark,"data=1,text=a ""string"" with quotes,va...",thrpt,ops/ms,208677.320609,208062.74007314906..209291.9011449821,198982.020454,194497.3197739599..203466.72113419612,-9695.300155,-4.646073,test.ParamBenchmark.mathBenchmark [da...,neg
test.ParamBenchmark.mathBenchmark,"data=2,text=a ""string"" with quotes,va...",thrpt,ops/ms,83625.738987,83088.20039438509..84163.27757924065,76487.508628,74595.04831374026..78379.96894260854,-7138.230359,-8.535925,test.ParamBenchmark.mathBenchmark [da...,neg
test.ParamBenchmark.mathBenchmark,"data=2,text=a ""string"" with quotes,va...",thrpt,ops/ms,83718.317770,83169.61975110027..84267.01578838592,77218.536895,75599.16372724227..78837.91006301393,-6499.780875,-7.763869,test.ParamBenchmark.mathBenchmark [da...,neg
test.ParamBenchmark.otherBenchmark,"data=1,text=a ""string"" with quotes,va...",thrpt,ops/ms,2540370.576263,2288100.709941338..2792640.4425841896,2312642.773644,2091779.438724716..2533506.1085633924,-227727.802619,-8.964354,test.ParamBenchmark.otherBenchmark [d...,neg
test.ParamBenchmark.otherBenchmark,"data=1,text=a ""string"" with quotes,va...",thrpt,ops/ms,2566945.341149,2360770.956540762..2773119.7257564147,2295922.938822,2089935.2366487498..2501910.6409945134,-271022.402327,-10.558168,test.ParamBenchmark.otherBenchmark [d...,neg
test.ParamBenchmark.otherBenchmark,"data=2,text=a ""string"" with quotes,va...",thrpt,ops/ms,2623681.403705,2551535.491785763..2695827.3156246524,2295223.364007,2115768.594299648..2474678.1337146433,-328458.039698,-12.518976,test.ParamBenchmark.otherBenchmark [d...,neg
test.ParamBenchmark.otherBenchmark,"data=2,text=a ""string"" with quotes,va...",thrpt,ops/ms,2540686.802958,2452532.053305992..2628841.5526102716,2295128.593211,2082445.0002591067..2507812.186162458,-245558.209747,-9.665033,test.ParamBenchmark.otherBenchmark [d...,neg


In [30]:
import org.jetbrains.kotlinx.kandy.util.color.Color
import org.jetbrains.letsPlot.core.spec.plotson.fill
import org.jetbrains.letsPlot.label.ggtitle
import org.jetbrains.letsPlot.scale.guideLegend
import org.jetbrains.letsPlot.scale.guides

// Now we can plot this data. First we create a basic plot just showing the difference in percent between all scores.
plotData.sortBy { diffScorePercentage }.plot {
    barsH {
        x(diffScorePercentage) {
            axis.name = "Diff %"
        }
        y(testLabel) {
            axis.name = ""
        }
        fillColor(barColor) {
            scale = categorical("neg" to Color.RED, "pos" to Color.GREEN)
            legend.type = LegendType.None
        }
        tooltips {
            line(diffScorePercentage, format = ".2f")
        }
    }
    layout {
        size = 800 to ((40 * plotData.size().nrow) + 100)
        style {
            global {
                title {
                    margin(10.0, 0.0)
                }
            }
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="NbEDDe"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"testLabel":["test.ParamBenchmark.otherBenchmark\n[data=2,text=a \"string\" with quotes,value=1]","test.ParamBenchmark.otherBenchmark\n[data=1,text=a \"string\" with quotes,value=2]","test.ParamBenchmark.otherBenchmark\n[data=2,text=a \"string\" with quotes,value=2]","test.CommonBenchmark.longBenchmark","test.ParamBenchmark.otherBenchmark\n[data=1,text=a \"string\" with quotes,value=1]","test.ParamBenchmark.mathBenchmark\n[data=2,text=a \"string\" with quotes,value=1]","test.JvmTestBenchmark.cosBenchmark","test.CommonBenchmark.longBlackholeBenchmark","test.ParamBenchmark.textContentCheck\n[data=2,text=a \"string\" with quotes,value=2]","test.JvmTestBenchmark.sqrtBenchmark","test.ParamBenchmark.mathBenchmark\n[data=2,text=a \"string\" with quotes,value=2]","test.CommonBenchmark.mathBenchmark","test.ParamBenchmark.textContentCheck\n[data=2,text=a \"string\" with quotes,value=1]","test.ParamBenchmark.textContentCheck\n[data=1,text=a \"string\" with quotes,value=2]","test.ParamBenchmark.mathBenchmark\n[data=1,text=a \"string\" with quotes,value=1]","test.InheritedBenchmark.inheritedBenchmark","test.ParamBenchmark.textContentCheck\n[data=1,text=a \"string\" with quotes,value=1]","test.nested.CommonBenchmark.mathBenchmark","test.InheritedBenchmark.baseBenchmark","test.ParamBenchmark.mathBenchmark\n[data=1,text=a \"string\" with quotes,value=2]"],
"diffScorePercentage":[-12.51897578853164,-10.55816802884034,-9.665032677835182,-9.519864802543674,-8.964353655588646,-8.535924997642304,-8.510342889653847,-8.379123205241132,-8.110200106903628,-8.00680778801588,-7.763869422808808,-7.640350293241543,-6.795872851346324,-5.878039657122179,-5.737570833099392,-5.704748465970368,-5.665144044984475,-5.621972075273389,-5.076124506088183,-4.646072762813873],
"barColor":["neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg"]
},
"ggsize":{
"width":800.0,
"height":900.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"Diff %",
"limits":[null,null]
},{
"aesthetic":"y",
"discrete":true,
"name":""
},{
"aesthetic":"fill",
"values":["#ee6666","#3ba272"],
"limits":["neg","pos"],
"guide":"none"
}],
"layers":[{
"mapping":{
"x":"diffScorePercentage",
"y":"testLabel",
"fill":"barColor"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"tooltips":{
"lines":["@|@{diffScorePercentage}"],
"formats":[{
"field":"diffScorePercentage",
"format":".2f"
}],
"disable_splitting":true
},
"data":{
}
}],
"theme":{
"title":{
"margin":[10.0,0.0,10.0,0.0],
"blank":false
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
},
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"diffScorePercentage"
},{
"type":"str",
"column":"testLabel"
},{
"type":"str",
"column":"barColor"
}]
},
"spec_id":"8"
};
 var containerDiv = document.getElementById("NbEDDe");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 800.0,
 height: 900.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -12 
 
 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 
 
 -8 
 
 
 
 
 
 
 
 
 -6 
 
 
 
 
 
 
 
 
 -4 
 
 
 
 
 
 
 
 
 -2 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 

In [31]:
// Just comparing the score values is a bit simplistic as the benchmark results are actually a range: score +/- error.
// So, instead of plotting all tests, we want to focus only on the benchmarks that looks "interesting". This is
// defined as any benchmark that differ so much that the benchmark ranges do not overlap, i.e., we no longer just
// look at only the score but consider the full error range.
//
// We still use the "score" to calculate the change in percent, but now on a filtered list
fun kotlin.ranges.ClosedFloatingPointRange<kotlin.Double>.overlaps(other: ClosedFloatingPointRange<kotlin.Double>): Boolean =
    this.start <= other.endInclusive && other.start <= this.endInclusive

val interestingBenchmarks = plotData.filter {
    !it.range.overlaps(it.range1)
}
interestingBenchmarks

name,params,mode,unit,score,range,score1,range1,diffScore,diffScorePercentage,testLabel,barColor
test.InheritedBenchmark.inheritedBenc...,,thrpt,ops/s,140537144.603648,1.3690787959174788E8..1.4416640961554...,132519854.002753,1.2936737342079675E8..1.3567233458470...,-8017290.600895,-5.704748,test.InheritedBenchmark.inheritedBenc...,neg
test.ParamBenchmark.mathBenchmark,"data=1,text=a ""string"" with quotes,va...",thrpt,ops/ms,208887.080329,207898.89918554123..209875.26147208977,196902.036134,191654.7423267418..202149.32994077116,-11985.044195,-5.737571,test.ParamBenchmark.mathBenchmark [da...,neg
test.ParamBenchmark.mathBenchmark,"data=1,text=a ""string"" with quotes,va...",thrpt,ops/ms,208677.320609,208062.74007314906..209291.9011449821,198982.020454,194497.3197739599..203466.72113419612,-9695.300155,-4.646073,test.ParamBenchmark.mathBenchmark [da...,neg
test.ParamBenchmark.mathBenchmark,"data=2,text=a ""string"" with quotes,va...",thrpt,ops/ms,83625.738987,83088.20039438509..84163.27757924065,76487.508628,74595.04831374026..78379.96894260854,-7138.230359,-8.535925,test.ParamBenchmark.mathBenchmark [da...,neg
test.ParamBenchmark.mathBenchmark,"data=2,text=a ""string"" with quotes,va...",thrpt,ops/ms,83718.317770,83169.61975110027..84267.01578838592,77218.536895,75599.16372724227..78837.91006301393,-6499.780875,-7.763869,test.ParamBenchmark.mathBenchmark [da...,neg
test.ParamBenchmark.otherBenchmark,"data=2,text=a ""string"" with quotes,va...",thrpt,ops/ms,2623681.403705,2551535.491785763..2695827.3156246524,2295223.364007,2115768.594299648..2474678.1337146433,-328458.039698,-12.518976,test.ParamBenchmark.otherBenchmark [d...,neg
test.ParamBenchmark.textContentCheck,"data=1,text=a ""string"" with quotes,va...",thrpt,ops/ms,147875.332398,147061.31984741875..148689.34494853206,139497.981811,135894.93943472698..143101.02418653402,-8377.350587,-5.665144,test.ParamBenchmark.textContentCheck ...,neg
test.ParamBenchmark.textContentCheck,"data=1,text=a ""string"" with quotes,va...",thrpt,ops/ms,148182.794650,147312.38741453286..149053.20188515788,139472.551215,135249.9703875965..143695.13204299458,-8710.243435,-5.878040,test.ParamBenchmark.textContentCheck ...,neg
test.ParamBenchmark.textContentCheck,"data=2,text=a ""string"" with quotes,va...",thrpt,ops/ms,148010.561236,147183.62506193086..148837.49741062557,137951.951688,133789.3696336104..142114.53374258292,-10058.609548,-6.795873,test.ParamBenchmark.textContentCheck ...,neg
test.ParamBenchmark.textContentCheck,"data=2,text=a ""string"" with quotes,va...",thrpt,ops/ms,147818.349576,146603.6094475876..149033.08970461992,135829.985631,131190.7048701939..140469.26639132493,-11988.363945,-8.110200,test.ParamBenchmark.textContentCheck ...,neg


In [32]:
// Now lets plot the interesting benchmarks, similar to before.
interestingBenchmarks.sortBy { diffScorePercentage }.plot {
    barsH {
        x(diffScorePercentage) {
            axis.name = "Diff %"
        }
        y(testLabel) {
            axis.name = ""
        }
        fillColor(barColor) {
            scale = categorical("neg" to Color.RED, "pos" to Color.GREEN)
            legend.type = LegendType.None
        }
        tooltips {
            line(diffScorePercentage, format = ".2f")
        }
    }
    layout {
        size = 800 to ((40 * interestingBenchmarks.size().nrow) + 100)
        style {
            global {
                title {
                    margin(10.0, 0.0)
                }
            }
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="LLZtLQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"testLabel":["test.ParamBenchmark.otherBenchmark\n[data=2,text=a \"string\" with quotes,value=1]","test.CommonBenchmark.longBenchmark","test.ParamBenchmark.mathBenchmark\n[data=2,text=a \"string\" with quotes,value=1]","test.JvmTestBenchmark.cosBenchmark","test.CommonBenchmark.longBlackholeBenchmark","test.ParamBenchmark.textContentCheck\n[data=2,text=a \"string\" with quotes,value=2]","test.JvmTestBenchmark.sqrtBenchmark","test.ParamBenchmark.mathBenchmark\n[data=2,text=a \"string\" with quotes,value=2]","test.CommonBenchmark.mathBenchmark","test.ParamBenchmark.textContentCheck\n[data=2,text=a \"string\" with quotes,value=1]","test.ParamBenchmark.textContentCheck\n[data=1,text=a \"string\" with quotes,value=2]","test.ParamBenchmark.mathBenchmark\n[data=1,text=a \"string\" with quotes,value=1]","test.InheritedBenchmark.inheritedBenchmark","test.ParamBenchmark.textContentCheck\n[data=1,text=a \"string\" with quotes,value=1]","test.nested.CommonBenchmark.mathBenchmark","test.ParamBenchmark.mathBenchmark\n[data=1,text=a \"string\" with quotes,value=2]"],
"diffScorePercentage":[-12.51897578853164,-9.519864802543674,-8.535924997642304,-8.510342889653847,-8.379123205241132,-8.110200106903628,-8.00680778801588,-7.763869422808808,-7.640350293241543,-6.795872851346324,-5.878039657122179,-5.737570833099392,-5.704748465970368,-5.665144044984475,-5.621972075273389,-4.646072762813873],
"barColor":["neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg","neg"]
},
"ggsize":{
"width":800.0,
"height":740.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"Diff %",
"limits":[null,null]
},{
"aesthetic":"y",
"discrete":true,
"name":""
},{
"aesthetic":"fill",
"values":["#ee6666","#3ba272"],
"limits":["neg","pos"],
"guide":"none"
}],
"layers":[{
"mapping":{
"x":"diffScorePercentage",
"y":"testLabel",
"fill":"barColor"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"tooltips":{
"lines":["@|@{diffScorePercentage}"],
"formats":[{
"field":"diffScorePercentage",
"format":".2f"
}],
"disable_splitting":true
},
"data":{
}
}],
"theme":{
"title":{
"margin":[10.0,0.0,10.0,0.0],
"blank":false
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
},
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"diffScorePercentage"
},{
"type":"str",
"column":"testLabel"
},{
"type":"str",
"column":"barColor"
}]
},
"spec_id":"11"
};
 var containerDiv = document.getElementById("LLZtLQ");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 800.0,
 height: 740.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -12 
 
 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 
 
 -8 
 
 
 
 
 
 
 
 
 -6 
 
 
 
 
 
 
 
 
 -4 
 
 
 
 
 
 
 
 
 -2 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 
 
 test.ParamBenchmark.otherBenchmark [data=2,text=a "string" with quotes,value=1] 
 
 
 
 
 
 
 test.CommonBenchmark.longBenchmark 
 
 
 
 
 
 
 test.ParamBenchmark.mathBenchmark [data=2,text=a "string" with quotes,value=1] 
 
 
 
 
 
 
 test.JvmTestBenchmark.cosBenchmark 
 
 
 
 
 
 
 test.CommonBenchmark.longBlackholeBenchmark 
 
 
 
 
 
 
 test.ParamBenchmark.textContentCheck [data=2,text=a "string" 